In [ ]:
import uuid

from datetime import datetime
from neo4j import GraphDatabase, exceptions
from numpy.core.records import record

In [45]:
URI = "bolt://127.0.0.1:7687"
AUTH = ("neo4j", "d1997225")  # 修改为你的密码
DATABASE = "chapter3"
driver = GraphDatabase.driver(URI, auth=AUTH)

In [ ]:
# Delete student
def delete_student(student_id):
    query = """
    MATCH (s: Student {studentId: $studentId})
    DETACH DELETE s
    """
    with driver.session(database=DATABASE) as session:
        session.run(query, studentId=student_id)
delete_student("S_0611C171")

In [40]:
# Create a new student
def create_student(name, student_id):
    query = """
    CREATE (s:Student {name: $name, studentId: $studentId} )
    RETURN s
    """
    with driver.session(database=DATABASE) as session:
        ifNew = True
        result = session.run(query,
                             name=name,
                             studentId = student_id)
        return result.single()[0]

student_name = "chloe"
student_id = f'S_{uuid.uuid4().hex[:8].upper()}'
create_student(student_name, student_id)

<Node element_id='4:8d5647cc-0abe-4dd1-9054-4758db8fd05f:102' labels=frozenset({'Student'}) properties={'studentId': 'S_C103FCBC', 'name': 'chloe'}>

In [41]:
def create_relationship(student_id, initial_level):
    query = """
    MATCH (s:Student {studentId: $studentId}), (c:Concept)
    MERGE (s)-[r:HAS_MASTERY]->(c)
    ON CREATE SET r.level = $level, r.last_updated = $last_updated
    RETURN s, collect(r) as mastery_relationships
    """
    with driver.session(database=DATABASE) as session:
        level = initial_level  # initial mastery level
        last_updated = datetime.now().isoformat()  # ISO formatted timestamp
        result = session.run(
            query,
            studentId=student_id,
            level=level,
            last_updated=last_updated
        )
        record = result.single()
        if record:
            return {
                "student": record["s"],
                "mastery_relationships": record["mastery_relationships"]
            }
        return None

In [42]:
initial_level = 0.1
result = create_relationship(student_id, initial_level)

In [60]:
import json
# then we need to deal the cold start issue
# here we randomly pick 5 points, later version we need to update it
def random_pick_5(num_questions=5):
    query="""
    MATCH (con: Concept)
    WITH con ORDER BY rand() LIMIT $num

    WITH con
    CALL (con) {
        MATCH (p:Problem)-[:TESTED]->(con)
        RETURN p
        LIMIT 1
    }

    RETURN p.id AS problemId,
           p.question AS question,
           p.options AS option,
           p.correct AS correct

    """
    driver = None
    try:
        driver = GraphDatabase.driver(URI, auth=AUTH)
        driver.verify_connectivity()

        with driver.session(database=DATABASE) as session:
            result = session.run(query, num = num_questions)
            questions = [record.data() for record in result]
            return questions

    except Exception as e:
        # 如果发生任何错误，打印错误信息并返回空列表
        print(f"An error occurred: {e}")
        return []

    finally:
        # 无论成功还是失败，最后都确保关闭数据库连接
        if driver is not None:
            driver.close()

question = random_pick_5()
question_json = json.dumps(question, ensure_ascii=False, indent=2)
print(question_json)
# then we can send this to frontend

[
  {
    "problemId": "test_478A638A",
    "question": "According to Dalton's Atomic Theory, which statement is a key principle?",
    "option": [
      "Atoms can be divided into smaller particles.",
      "Atoms of one element can be converted into atoms of another element.",
      "All matter is made up of tiny particles called atoms.",
      "Atoms of the same element can have different masses."
    ],
    "correct": 2
  },
  {
    "problemId": "test_42510643",
    "question": "The central core of an atom, containing protons and neutrons, is called the:",
    "option": [
      "Electron cloud",
      "Nucleus",
      "Energy level",
      "Valence shell"
    ],
    "correct": 1
  },
  {
    "problemId": "test_E8CB13E6",
    "question": "A Lewis structure uses dots to represent which type of electrons?",
    "option": [
      "All electrons in an atom",
      "Only the electrons in the first energy level",
      "Only the valence electrons",
      "Only the core electrons"
    ],
 

In [62]:
# assume we already had all the result of these problems
# 下一步是将这5个简单的“对/错”结果，转化为学生在知识图谱中的初始“知识画像”，然后立即给出第一次个性化推荐。
student_id = student_id
test_results = [
    {'problemId': 'test_478A638A', 'isCorrect': True},
    {'problemId': 'test_42510643', 'isCorrect': False},
    {'problemId': 'test_E8CB13E6', 'isCorrect': True},
    {'problemId': 'test_031FAF92', 'isCorrect': False},
    {'problemId': 'test_50C4294C', 'isCorrect': True}
]

您的下一步是将这5个简单的“对/错”结果，转化为学生在知识图谱中的初始“知识画像”，然后立即给出第一次个性化推荐。## Check the mastery level
Assume I have a user call "Chloe", who can be sign up today or probably few days ago, so the date cannot be the criteria, so we check the mastery level, if all

In [31]:
def check_mastery_zero(records):

    all_zero = True
    assert records != []

    for record in records:
        concept = record['concept_name']
        mastery_level = record['mastery']

        print(f"Concept: {concept}, Mastery: {mastery_level}")
        if mastery_level != 0:
            all_zero = False
    return all_zero


In [34]:
def check_mastery_zero(records):

    all_zero = True
    assert records != []

    for record in records:
        concept = record['concept_name']
        mastery_level = record['mastery']

        print(f"Concept: {concept}, Mastery: {mastery_level}")
        if mastery_level != 0:
            all_zero = False
    return all_zero

def check_mastery(user_name):
    """
    Checks if the user has all mastery levels at zero.
    If not, sets user.new = 'FALSE'.
    """

    query_fetch = """
    MATCH (c:User {name: $name})-[r:HAS_MASTERY]->(con:Concept)
    RETURN r.mastery AS mastery, con.name AS concept_name
    """
    query_update = """
    MATCH (a:User {name: $name})
    SET a.new = {$new_student}
    """
    params_fetch = {"name" : user_name}
    params_update = {"name" : user_name, "new_student": "FALSE"}

    with driver.session(database=DATABASE) as session:
        result = session.run(query_fetch, params_fetch)
        records = list(result)
        if check_mastery_zero(records):
            pass
        else:
            session.run(query_update, params_update)


In [35]:
check_mastery("Chloe")

Concept: Atom, Mastery: 0
Concept: Dalton's Atomic Theory, Mastery: 0
Concept: Modern Atomic Theory, Mastery: 0
Concept: Subatomic Particle, Mastery: 0
Concept: Proton, Mastery: 0
Concept: Neutron, Mastery: 0
Concept: Electron, Mastery: 0
Concept: Nucleus, Mastery: 0
Concept: Atomic Number (Z), Mastery: 0
Concept: Mass Number (A), Mastery: 0
Concept: Isotope, Mastery: 0
Concept: Radioisotope, Mastery: 0
Concept: Periodic Table, Mastery: 0
Concept: Periodic Law, Mastery: 0
Concept: Energy Level, Mastery: 0
Concept: Valence Electron, Mastery: 0
Concept: Lewis Structure, Mastery: 0
Concept: Stable Octet, Mastery: 0
Concept: Periodic Trend, Mastery: 0
Concept: Atomic Radius, Mastery: 0
Concept: Ion, Mastery: 0
Concept: Cation, Mastery: 0
Concept: Anion, Mastery: 0
Concept: Ionization Energy, Mastery: 0
Concept: Electron Affinity, Mastery: 0


## Now we know that "Chloe" is the new student
Next step is to random push 3 problems to her

In [57]:
# first, check if "Chloe" is a new student
# assume she is, random choose 3 problems for her

def random_fetch_concept(user_name, n=3):
    """
    Fetch `n` random Concept nodes that the user has HAS_MASTERY relationships with,
    along with all Problem nodes linked via TESTED (Problem -> Concept).
    Returns a list of dictionaries with concept name, mastery, and problems list.
    """
    query_fetch = """
    MATCH (c:User {name: $user_name})-[r:HAS_MASTERY]->(con:Concept)
    OPTIONAL MATCH (p:Problem)-[:TESTED]->(con)
    RETURN con.name AS concept_name, r.mastery AS mastery, collect(p) AS problems
    ORDER BY rand()
    LIMIT $limit
    """
    params_fetch = {"user_name": user_name, "limit": n}

    with driver.session(database=DATABASE) as session:
        result = session.run(query_fetch, params_fetch)
        concepts = []
        for record in result:
            # Convert each Problem node to a dictionary of its properties
            problems_list = [dict(p) for p in record["problems"] if p is not None]
            concepts.append({
                "concept_name": record["concept_name"],
                "mastery": record["mastery"],
                "problems": problems_list
            })
    return concepts



In [58]:
random_fetch_concept("Chloe", 3)

[{'concept_name': 'Isotope',
  'mastery': 0,
  'problems': [{'difficulty': 'easy',
    'question': 'Carbon-12 and Carbon-14 are examples of:',
    'correct': 1,
    'options': ['Ions', 'Isotopes', 'Isomers', 'Cations'],
    'id': 'test_031FAF92'},
   {'difficulty': 'medium',
    'question': 'What do isotopes of an element have in common?',
    'correct': 2,
    'options': ['Mass number',
     'Number of neutrons',
     'Atomic number',
     'Atomic mass'],
    'id': 'test_94AC2D28'},
   {'difficulty': 'medium',
    'question': 'Which of these pairs represents isotopes of the same element?',
    'correct': 2,
    'options': ['${}^{14}_6$C and ${}^{14}_7$N',
     '${}^{39}_{19}$K and ${}^{40}_{20}$Ca',
     '${}^{235}_{92}$U and ${}^{238}_{92}$U',
     '${}^{16}_8$O and ${}^{18}_9$F'],
    'id': 'test_35937AE0'},
   {'difficulty': 'easy',
    'question': 'What is the mass number of an isotope with 20 protons and 22 neutrons?',
    'correct': 3,
    'options': ['20', '22', '40', '42'],
  

In [10]:
# Create a new student
def create_student(name):
    query = """
    CREATE (s: Student (name: $name, studentId: $studentId, ifNew=$ifNew)
    RETURN s
    """
    with driver.session() as session:
        ifNew = True
        result = session.run(query,
                             name=name,
                             studentId = f'S_{uuid.uuid4().hex[:8].upper()}',
                             ifNew=ifNew)
        return result.single()[0]


create_student("chloe")

ClientError: {code: Neo.ClientError.Database.DatabaseNotFound} {message: Database does not exist. Database name: 'neo4j'.}